In [1]:
# !pip install tensorflow_gnn

In [2]:
import pdb # use pdb.set_trace() for debuging
import tensorflow     as tf
from   tensorflow import keras
from   keras      import layers
import tensorflow_gnn as tfgnn

import plotly.graph_objects as go
from   plotly.subplots  import make_subplots
import plotly.io            as pio

import numpy  as np
import pandas as pd

from GraphFun.generate_graph_Econn      import generate_graph_Econn
from GraphFun.plotGraph                 import plotGraph
from GraphFun.make_edge_bidirectional   import make_edge_bidirectional
from NormalisingFun.normalise_dataframe import normalise_dataframe

from EdgesetFun.edgeset_function_1_0 import generate_graph_multi_sets
from EdgesetFun.label_function_1_0   import label_function

In [19]:
# Edgeset Generation

n_nodes_list_train  = [15,15]
n_edges_list_train  = [(1, 2),(1,2)]
n_conn_list_train   = [[1,1],[2,2],[1,2]]
n_edges_ij_train    = [2,2]

n_nodes_list_test  = [15,15]
n_edges_list_test  = [(1, 2),(1,2)]
n_conn_list_test   = [[1,1],[2,2],[1,2]]
n_edges_ij_test    = [2,2]

n_nodes_list_val  = [15,15]
n_edges_list_val  = [(1, 2),(1,2)]
n_conn_list_val   = [[1,1],[2,2],[1,2]]
n_edges_ij_val    = [2,2]

bidirectional = True

df_list_train, edge_df_train = generate_graph_multi_sets(n_nodes_list_train, n_edges_list_train, n_conn_list_train, n_edges_ij_train, bidirectional)
df_list_test, edge_df_test   = generate_graph_multi_sets(n_nodes_list_test, n_edges_list_test, n_conn_list_test, n_edges_ij_test, bidirectional)
df_list_val, edge_df_val     = generate_graph_multi_sets(n_nodes_list_val, n_edges_list_val, n_conn_list_val, n_edges_ij_val, bidirectional)

# Nodeset Generation
node_df_train = label_function(n_nodes_list_train, df_list_train)
node_df_test  = label_function(n_nodes_list_test, df_list_test)
node_df_val   = label_function(n_nodes_list_val, df_list_val)

In [6]:
def edge_features(edge_df):
  features_len = len(edge_df)
  features_array = 1 + 0.5*np.random.uniform(-1, 1, size=[features_len,1])
  features_df = pd.DataFrame(features_array, columns=['edge_feature'])
  edge_df = pd.concat([features_df, edge_df], axis = 1)
  edge_df = edge_df[['edge_feature','source','target','set','source_nodeset','target_nodeset']]
  return edge_df


edge_df_train = edge_features(edge_df_train)
edge_df_test = edge_features(edge_df_test)
edge_df_val = edge_features(edge_df_val)

In [9]:
NormTYPE = 'STD'
node_df_train,  node_df_train_mean, node_df_train_std   = normalise_dataframe(node_df_train, NormTYPE)
edge_df_train,  edge_df_train_mean, edge_df_train_std   = normalise_dataframe(edge_df_train, NormTYPE)

# different from conventional database, here the normalisation is based on the training and
# applied equally to the other graphs to avoid diffferent normalisations
for iloc in range(len(node_df_val.keys())):
    my_key = node_df_train.keys()[iloc]
    if node_df_val[my_key].dtype != 'bool' and my_key!='set' and my_key!='ids':
        node_df_val.iloc[:,iloc]  = (node_df_val.iloc[:,iloc]-node_df_train_mean[iloc])/node_df_train_std[iloc]
        node_df_test.iloc[:,iloc] = (node_df_test.iloc[:,iloc]-node_df_train_mean[iloc])/node_df_train_std[iloc]

for iloc in range(len(edge_df_train.keys())):
    my_key = edge_df_train.keys()[iloc]
    if edge_df_train[my_key].dtype != 'bool' and my_key!='source' and my_key!='target' and my_key!='set' and my_key!='source_nodeset' and my_key!='target_nodeset':
        edge_df_val.iloc[:,iloc]  = (edge_df_val.iloc[:,iloc]-edge_df_train_mean[iloc])/edge_df_train_std[iloc]
        edge_df_test.iloc[:,iloc] = (edge_df_test.iloc[:,iloc]-edge_df_train_mean[iloc])/edge_df_train_std[iloc]

# edge_df_train = make_edge_bidirectional(edge_df_train)
# edge_df_val   = make_edge_bidirectional(edge_df_val)
# edge_df_test  = make_edge_bidirectional(edge_df_test)

if len(node_df_train)<=100:
    plotGraph(node_df_train, edge_df_train)


print('---------------------')
print(np.mean(node_df_train['node_feature'].values))
print(np.std(node_df_train['node_feature'].values))

print(np.mean(node_df_val['node_feature'].values))
print(np.std(node_df_val['node_feature'].values))

print(np.mean(node_df_test['node_feature'].values))
print(np.std(node_df_test['node_feature'].values))

print('---------------------')
print(np.mean(edge_df_train['edge_feature'].values))
print(np.std(edge_df_train['edge_feature'].values))

print(np.mean(edge_df_val['edge_feature'].values))
print(np.std(edge_df_val['edge_feature'].values))

print(np.mean(edge_df_test['edge_feature'].values))
print(np.std(edge_df_test['edge_feature'].values))


Issue with the function below, I think. I want a way to inspect the tensors created by this function.

In [11]:
def create_graph_tensor(node_df, edge_df):

    id_ns1  = node_df['set']==1
    id_ns2  = node_df['set']==2
    id_es1  = edge_df['set']==1
    id_es2  = edge_df['set']==2
    id_es12 = edge_df['set']==12

    # pdb.set_trace()

    graph_tensor = tfgnn.GraphTensor.from_pieces(
        node_sets = {
            "node_set1": tfgnn.NodeSet.from_fields(
                sizes = [sum(id_ns1)],
                features ={
                            'node_feature': np.array(node_df['node_feature'][id_ns1], dtype='float32').reshape(sum(id_ns1),1), # input feature
                            'label':        np.array(node_df['label'][id_ns1], dtype='float32').reshape(sum(id_ns1),1),        # label
                          }),

            "node_set2": tfgnn.NodeSet.from_fields(
                sizes = [sum(id_ns2)],
                features ={
                            'node_feature': np.array(node_df['node_feature'][id_ns2], dtype='float32').reshape(sum(id_ns2),1), # input feature
                            'label':        np.array(node_df['label'][id_ns2], dtype='float32').reshape(sum(id_ns2),1),        # label
                          })},

        edge_sets ={
            "edge_set1": tfgnn.EdgeSet.from_fields(
                sizes = [sum(id_es1)],
                features = {
                    'edge_feature': np.array(edge_df['edge_feature'][id_es1],  dtype='float32').reshape(sum(id_es1),1),        # inputs
                    },
                adjacency = tfgnn.Adjacency.from_indices(
                    source = ("node_set1", np.array(edge_df['source'][id_es1], dtype='int32')),
                    target = ("node_set1", np.array(edge_df['target'][id_es1], dtype='int32')))),

            "edge_set2": tfgnn.EdgeSet.from_fields(
                sizes = [sum(id_es2)],
                features = {
                    'edge_feature': np.array(edge_df['edge_feature'][id_es2],  dtype='float32').reshape(sum(id_es2),1),        # inputs
                    },
                adjacency = tfgnn.Adjacency.from_indices(
                    source = ("node_set2", np.array(edge_df['source'][id_es2], dtype='int32')),
                    target = ("node_set2", np.array(edge_df['target'][id_es2], dtype='int32'))
                    )),

            "edge_set12": tfgnn.EdgeSet.from_fields(
                sizes = [sum(id_es12)],
                features = {
                    'edge_feature': np.array(edge_df['edge_feature'][id_es12],  dtype='float32').reshape(sum(id_es12),1),        # inputs
                    },
                adjacency = tfgnn.Adjacency.from_indices(
                    source = ("node_set1", np.array(edge_df['source'][id_es12], dtype='int32')),
                    target = ("node_set2", np.array(edge_df['target'][id_es12], dtype='int32'))
                    ))
      })
    return graph_tensor

train_tensor = create_graph_tensor(node_df_train, edge_df_train)
val_tensor   = create_graph_tensor(node_df_val,   edge_df_val)
test_tensor  = create_graph_tensor(node_df_test,  edge_df_test)

In [12]:
def node_batch_merge(graph):
    graph              = graph.merge_batch_to_components()
    node_features_set1 = graph.node_sets['node_set1'].get_features_dict()  # returns a dict of tensors
    edge_features_set1 = graph.edge_sets['edge_set1'].get_features_dict()  # returns a dict of tensors

    node_features_set2 = graph.node_sets['node_set2'].get_features_dict()  # returns a dict of tensors
    edge_features_set2 = graph.edge_sets['edge_set2'].get_features_dict()  # returns a dict of tensors

    # label = node_features_set1.pop('label')
    # _ = node_features_set2.pop('label')

    # label = node_features_set2.pop('label')
    # _ = node_features_set1.pop('label')

    label_1 = node_features_set1.pop('label')
    label_2 = node_features_set2.pop('label')
    label = tf.concat([label_1, label_2], axis=0)

    new_graph = graph.replace_features(
        node_sets={'node_set1':node_features_set1,
                   'node_set2':node_features_set2
                   },
        edge_sets={'edge_set1':edge_features_set1,
                   'edge_set2':edge_features_set2}
        )

    return new_graph, label

batch_size = 32
def create_dataset(graph,function):
    dataset = tf.data.Dataset.from_tensors(graph)   # 1
    dataset = dataset.batch(batch_size)                     # 2
    return dataset.map(function)                    # 3


#Node Datasets
train_node_dataset = create_dataset(train_tensor,node_batch_merge)
val_node_dataset   = create_dataset(val_tensor,node_batch_merge)
test_node_dataset  = create_dataset(test_tensor,node_batch_merge)

In [13]:
graph_spec  = train_node_dataset.element_spec[0]
input_graph = tf.keras.layers.Input(type_spec=graph_spec)

In [ ]:
n_mapping_neurons = 0
def set_initial_node_state(node_set, node_set_name):
    if n_mapping_neurons == 0:
        features = node_set['node_feature']  # works (no mapping)
    else:
        features = tf.keras.layers.Dense(n_mapping_neurons,activation="relu")(node_set['node_feature']) # works (1 layer mapping)
        # features = tf.keras.layers.Dense(8,activation="relu")(tf.keras.layers.Dense(8,activation="relu")(node_set['node_feature'])) # works (2 layer mapping)
    return features
    # Or we can also have multiple features
    # original_features = node_set['x_norm']
    # embeded_features  = tf.keras.layers.Dense(8,activation="relu")(node_set['x_norm'])
    # return tf.keras.layers.Concatenate()([original_features, embeded_features])



def set_initial_edge_state(edge_set, edge_set_name):
    if n_mapping_neurons == 0:
        features = edge_set['edge_feature'] # no mapping
    else:
        features = tf.keras.layers.Dense(8,activation="relu")(edge_set['edge_feature'])

    return features


graph = tfgnn.keras.layers.MapFeatures(
    node_sets_fn=set_initial_node_state,
    edge_sets_fn=set_initial_edge_state
)(input_graph)  # KerasTensor


In [15]:
def dense_layer(units=1, l2_reg=0.1, activation='relu'):
    regularizer = tf.keras.regularizers.l2(l2_reg)
    return tf.keras.Sequential([
        tf.keras.layers.Dense(units,
                              kernel_regularizer=regularizer,
                              bias_regularizer=regularizer,
                              activation=activation)])


def n_dense_layer(n_layers=1, units=8,l2_reg=0.01, dropout=0, activation='relu'):
    regularizer = tf.keras.regularizers.l2(l2_reg)
    model = tf.keras.Sequential()
    for i in range(n_layers):
        model.add(layers.Dense(units, activation=activation, kernel_regularizer=regularizer ))

        if dropout>0:
            model.add(layers.Dropout(dropout))

    return model

In [ ]:
n_layers            = 1
l2_reg              = 1e-8
dropout             = 0 # between 0 and 1
activation          = 'relu'
n_neurons_per_layer = 1
n_concat_neurons    = 1


graph_updates = 1
for i in range(graph_updates):
    graph = tfgnn.keras.layers.GraphUpdate(
        # edge update
        # for each edge, concatenate Target and Source node features
        # then pass it through a dense layer, which has n_concat_neurons output
        #
        # node update
        # for each node, if connected to multiple edges, the results of each updated edge features are pooled (summed)
        # then concatenated with the current node feature and the results is pass through another concat layer
        #
        edge_sets = {
                    'edge_set1':
                    tfgnn.keras.layers.EdgeSetUpdate(
                                            next_state=tfgnn.keras.layers.NextStateFromConcat(tf.keras.layers.Dense(n_concat_neurons)),
                                            edge_input_feature      = tfgnn.HIDDEN_STATE,
                                            node_input_tags         = (tfgnn.SOURCE, tfgnn.TARGET),
                                            node_input_feature      = tfgnn.HIDDEN_STATE,
                                            context_input_feature   = None      ),


                    'edge_set2':
                    tfgnn.keras.layers.EdgeSetUpdate(
                                            next_state=tfgnn.keras.layers.NextStateFromConcat(tf.keras.layers.Dense(n_concat_neurons)),
                                            edge_input_feature      = tfgnn.HIDDEN_STATE,
                                            node_input_tags         = (tfgnn.SOURCE, tfgnn.TARGET),
                                            node_input_feature      = tfgnn.HIDDEN_STATE,
                                            context_input_feature   = None      )
                    },

       node_sets={
                   "node_set1":
                  tfgnn.keras.layers.NodeSetUpdate(
                        {"edge_set1": tfgnn.keras.layers.Pool(tfgnn.SOURCE, "sum") },
                        tfgnn.keras.layers.NextStateFromConcat(tf.keras.layers.Dense(n_concat_neurons))),

                   "node_set2":
                  tfgnn.keras.layers.NodeSetUpdate(
                        {"edge_set2": tfgnn.keras.layers.Pool(tfgnn.SOURCE, "sum") },
                        tfgnn.keras.layers.NextStateFromConcat(tf.keras.layers.Dense(n_concat_neurons)))

                  })(graph) #start here

    # output = tf.keras.layers.Dense(1,activation='linear')(graph.node_sets["node_set1"][tfgnn.HIDDEN_STATE])
    # output = tf.keras.layers.Dense(1,activation='linear')(graph.node_sets["node_set2"][tfgnn.HIDDEN_STATE])

    output = tf.keras.layers.Dense(1,activation='linear')\
                ( tf.concat(
                              [graph.node_sets["node_set1"][tfgnn.HIDDEN_STATE] ,
                              graph.node_sets["node_set2"][tfgnn.HIDDEN_STATE] ], axis=0 ))


node_model = tf.keras.Model(input_graph, output)


##### compile
learning_rate = 0.02
node_model.compile(
    tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss = tf.keras.losses.MeanSquaredError()
)

node_model.summary()


es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',mode='min',verbose=1,
        patience=10,restore_best_weights=True)


history = node_model.fit(train_node_dataset.repeat(),
                           validation_data=val_node_dataset,
                           steps_per_epoch=10,
                           epochs=125,
                           callbacks=[es])


loss = node_model.evaluate(train_node_dataset)
loss = node_model.evaluate(val_node_dataset)

import matplotlib.pyplot as plt
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  # plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)

plot_loss(history)

# True Vs Prediction plot
y    = np.array(node_df_train['label'])
yhat = node_model.predict(train_node_dataset).flatten()
Err  = abs((y-yhat))

y_val    = np.array(node_df_val['label'])
yhat_val = node_model.predict(val_node_dataset).flatten()
Err      = abs((y_val-yhat_val))

fig = go.Figure()

fig.add_trace(go.Scatter( x=[-15,15], y=[-15,15], mode = 'lines', line=dict(color='black')))

fig.add_trace(go.Scatter(x=y.flatten(),
                         y=yhat.flatten(),
                         mode = 'markers',
                         line    = dict(color='blue'),
                         marker  = dict(color='blue'),
                         name= 'label ',
                         ))

fig.add_trace(go.Scatter(x=y_val.flatten(),
                         y=yhat_val.flatten(),
                         mode = 'markers',
                         line    = dict(color='red'),
                         marker  = dict(color='red'),
                         name= 'label ',
                         ))
fig.update_xaxes( title_text="True label")
fig.update_yaxes( title_text="Predicted label")

In [ ]:
node_model.get_weights()

weights = node_model.weights
for weight in weights:
    weight_values = weight.numpy().flatten()
    layer_name    = weight.name.split('/')[0]  # Extract the layer name from the weight name
    print(f"Weight name: {weight.name}, Layer name: {layer_name}, Weight Values: {weight_values}")